In [1]:
import torch
import torchvision

In [2]:
import os

proxy = 'http://proxy-k.ecl.ntt.co.jp:8080'

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print("device", device)

device cuda:0


In [4]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.manual_seed(random_seed)

In [5]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [9]:
network = Net()
network.to(device)
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [13]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [29]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    
    data = data.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target).to(device)
    
    loss.backward()
    optimizer.step()

    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.cpu().item()))
      train_losses.append(loss.cpu().item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      os.makedirs("./results", exist_ok=True)
      torch.save(network.state_dict(), './results/model.pth')
      torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [30]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:

      data = data.to(device)
      target = target.to(device)

      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).cpu().item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
      
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [31]:
# test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

C:\Users\1001002996\AppData\Local\Temp\ipykernel_6068\161431047.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.282855
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.294821
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.299128
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.377444
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.317626
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.248219
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.203889
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.140966
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.403411
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.188657
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.251372
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.391372
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.202174
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.249997
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.361188
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.372688
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.189202
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.208028
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.308843
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.266153
Train Epoch: 1 [12800/60000 (